<a href="https://colab.research.google.com/github/estefanabermeo/RegresionNoLineal_RegresionLogistica/blob/main/A7_2_Gastos2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ACTIVIDAD 7.2 REGRESIÓN LOGÍSTICA - GASTOS 2O21**
EQUIPO 10:
* Estefana Bermeo Severiano - A01367558
* Miguel Saúl Fernández Ávalos - A01707491
* Mariel Quetzali Fernández Montes - A0170779
* Christian Jesús Soto-Vieyra Gil - A01707759

---

In [23]:
#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [24]:
# Leemos el archivo ya limpio
gastos21 = pd.read_csv("gastos2021_limpio.csv")
gastos21.head(5)

,Unnamed: 0,FECHA,FOLIO,UUID,RFC,PROVEEDOR,TIPO GASTO,DESCRIPCION,MP,TC,IMPORTE,IVA,RET ISR,RET IVA,TOTAL MX,TOTAL SAT,TIPO,STATUS,POLIZA
0,0,2021-01-01,0001010100750847116902,71B4C8B9-8DAA-47CC-AD47-9DD90AA5AF6B,BNM840515VB1,"BANCO NACIONAL DE MEXICO, S.A.",COMISION BANCARIA,COMISION COBRADA,PUE,1.0,490.00,78.40,0.0,0.0,568.4,568.4,I,Vigente,0
1,1,2021-01-01,0001010100752163714102,B8AEDE83-CC76-404C-A611-6B5A87BA561F,BNM840515VB1,"BANCO NACIONAL DE MEXICO, S.A.",COMISION BANCARIA,COMISION COBRADA,PUE,1.0,345.00,55.20,0.0,0.0,400.2,400.2,I,Vigente,0
2,2,2021-01-01,19788,19B9B2C2-DF58-E24D-B43A-3FC4D5A929E7,ESI9612311E7,"ELECTRONICOS Y SIMILARES, S.A. DE C.V.",COMPRAS,MOTO E7 PLUS AZUL 64GB 4RAM,PUE,1.0,3447.41,551.59,0.0,0.0,3999.0,3999.0,I,Vigente,0
3,3,2021-01-01,211781,EAB26414-DD9A-4E8F-9F5C-05FE24D98E48,BLE1512082R9,BEPENSA LEASING S.A. DE C.V.,ARRENDAMIENTO FINANCIERO,AA RENTA EXIGIBLE CL0000006876 6/36 de 03/12/2...,PPD,1.0,5551.38,888.22,0.0,0.0,6439.6,6439.6,I,Vigente,0
4,4,2021-01-01,211782,C16F79B5-FB35-4BFC-AF84-708B24F500BB,BLE1512082R9,BEPENSA LEASING S.A. DE C.V.,ARRENDAMIENTO FINANCIERO,AA RENTA EXIGIBLE CL0000006877 6/36 de 03/12/2...,PPD,1.0,5551.98,888.32,0.0,0.0,6440.3,6440.3,I,Vigente,0


In [25]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(gastos21['TIPO GASTO'])
unico

array(['ALIMENTOS', 'ARRENDAMIENTO FINANCIERO', 'ARRENDAMIENTO LOCAL',
       'BOTIQUIN', 'CALIBRACIONES', 'CAPACITACION', 'COMBUSTIBLE ',
       'COMISION BANCARIA', 'COMISION MIXTA', 'COMPRA COMPUTADORA ',
       'COMPRA TABLET ', 'COMPRAS', 'COMPRAS ', 'DERECHOS',
       'ENERGIA ELECTRICA', 'FLETE', 'FLETES', 'GASOLINA',
       'GASTOS IMPORTACION ', 'GERENCIA ADMON', 'GERENCIA VTAS',
       'HONORARIOS CONTABLES', 'HONORARIOS JURIDICOS', 'HONORARIOS PF',
       'HONORARIOS PM', 'HOSPEDAJE', 'HOSPEDAJE ', 'IMSS/INFONAVIT',
       'MAQUILAS', 'MAQUILAS GIC', 'MAQUINARIA', 'MEDICOS', 'MENSAJERIA',
       'MONITOREO CAMARAS ADT', 'MTTO COMPUTO', 'MTTO LOCAL',
       'MTTO MAQUINARIA', 'MTTO OFICINA ', 'MTTO TRANSPORTE',
       'NO DEDUCIBLE', 'PAPELERIA', 'PAQUETERIA', 'PEAJES', 'PUBLICIDAD',
       'RENTA COPIADORA', 'RENTA OFICINA ', 'SANITIZACION', 'SEGUROS',
       'SUELDOS DIRECCION', 'SUELDOS PRODUCCION', 'SUELDOS VENTAS',
       'SUSCRIPCIONES', 'TELEFONO', 'TRANSPORTE', 'UNIFO

Convertir la variable TIPO GASTO en una variable dicotómica

In [27]:
#Convierto una variable a dicotómica 
gastos21['TIPO GASTO']= gastos21['TIPO GASTO'].replace(['COMPRAS '], "COMPRAS")

In [28]:
#Convierto una variable a dicotómica 
gastos21['TIPO GASTO']= gastos21['TIPO GASTO'].replace(['ALIMENTOS', 'ARRENDAMIENTO FINANCIERO', 'ARRENDAMIENTO LOCAL',
       'BOTIQUIN', 'CALIBRACIONES', 'CAPACITACION', 'COMBUSTIBLE ',
       'COMISION BANCARIA', 'COMISION MIXTA', 'COMPRA COMPUTADORA ',
       'COMPRA TABLET ', 'DERECHOS',
       'ENERGIA ELECTRICA', 'FLETE', 'FLETES', 'GASOLINA',
       'GASTOS IMPORTACION ', 'GERENCIA ADMON', 'GERENCIA VTAS',
       'HONORARIOS CONTABLES', 'HONORARIOS JURIDICOS', 'HONORARIOS PF',
       'HONORARIOS PM', 'HOSPEDAJE', 'HOSPEDAJE ', 'IMSS/INFONAVIT',
       'MAQUILAS', 'MAQUILAS GIC', 'MAQUINARIA', 'MEDICOS', 'MENSAJERIA',
       'MONITOREO CAMARAS ADT', 'MTTO COMPUTO', 'MTTO LOCAL',
       'MTTO MAQUINARIA', 'MTTO OFICINA ', 'MTTO TRANSPORTE',
       'NO DEDUCIBLE', 'PAPELERIA', 'PAQUETERIA', 'PEAJES', 'PUBLICIDAD',
       'RENTA COPIADORA', 'RENTA OFICINA ', 'SANITIZACION', 'SEGUROS',
       'SUELDOS DIRECCION', 'SUELDOS PRODUCCION', 'SUELDOS VENTAS',
       'SUSCRIPCIONES', 'TELEFONO', 'TRANSPORTE', 'UNIFORME',
       'VALES DESPENSA'], "OTROS GASTOS")
gastos21

,Unnamed: 0,FECHA,FOLIO,UUID,RFC,PROVEEDOR,TIPO GASTO,DESCRIPCION,MP,TC,IMPORTE,IVA,RET ISR,RET IVA,TOTAL MX,TOTAL SAT,TIPO,STATUS,POLIZA
0,0,2021-01-01,0001010100750847116902,71B4C8B9-8DAA-47CC-AD47-9DD90AA5AF6B,BNM840515VB1,"BANCO NACIONAL DE MEXICO, S.A.",OTROS GASTOS,COMISION COBRADA,PUE,1.0,490.00,78.40,0.0,0.0,568.40,568.40,I,Vigente,0
1,1,2021-01-01,0001010100752163714102,B8AEDE83-CC76-404C-A611-6B5A87BA561F,BNM840515VB1,"BANCO NACIONAL DE MEXICO, S.A.",OTROS GASTOS,COMISION COBRADA,PUE,1.0,345.00,55.20,0.0,0.0,400.20,400.20,I,Vigente,0
2,2,2021-01-01,19788,19B9B2C2-DF58-E24D-B43A-3FC4D5A929E7,ESI9612311E7,"ELECTRONICOS Y SIMILARES, S.A. DE C.V.",COMPRAS,MOTO E7 PLUS AZUL 64GB 4RAM,PUE,1.0,3447.41,551.59,0.0,0.0,3999.00,3999.00,I,Vigente,0
3,3,2021-01-01,211781,EAB26414-DD9A-4E8F-9F5C-05FE24D98E48,BLE1512082R9,BEPENSA LEASING S.A. DE C.V.,OTROS GASTOS,AA RENTA EXIGIBLE CL0000006876 6/36 de 03/12/2...,PPD,1.0,5551.38,888.22,0.0,0.0,6439.60,6439.60,I,Vigente,0
4,4,2021-01-01,211782,C16F79B5-FB35-4BFC-AF84-708B24F500BB,BLE1512082R9,BEPENSA LEASING S.A. DE C.V.,OTROS GASTOS,AA RENTA EXIGIBLE CL0000006877 6/36 de 03/12/2...,PPD,1.0,5551.98,888.32,0.0,0.0,6440.30,6440.30,I,Vigente,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3030,3030,2021-12-01,127043,83F82167-0D14-144E-9D76-045C95F25DDD,PGM881128QIA,"PROMOCIONES GRAFICAS MEXICANAS, S.A. DE C.V.",OTROS GASTOS,Pago,--,1.0,0.00,0.00,0.0,0.0,0.00,0.00,P,Vigente,--
3031,3031,2021-12-01,11740,B1C60AE0-02BC-4A33-9513-A280F72B5D06,SME0510105C7,SPECTRIS MEXICO S DE RL DE CV,COMPRAS,INSULATED T/C WIRE,PUE,1.0,0.00,0.00,0.0,0.0,0.00,0.00,I,Vigente,--
3032,3032,2021-12-01,1092957089,255F4A25-5753-4208-8F6B-0B59E5AE4741,BMN930209927,BANCO MERCANTIL DEL NORTE SA INSTITUCION DE BA...,OTROS GASTOS,SERVICIOS BANCARIOS,PUE,1.0,22826.46,705.58,0.0,0.0,23532.04,23532.04,I,Vigente,--
3033,3033,2021-12-01,747826,D8226E4F-9241-44C3-86A4-CDE9BC932528,BIN940223KE0,"BANCO INVEX, S.A., INSTITUCION DE BANCA MULTIP...",OTROS GASTOS,COMISIONES E INTERESES,PUE,1.0,0.00,0.00,0.0,0.0,0.00,0.00,I,Vigente,--


Analizar los siguientes casos de correlación logística

In [29]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= gastos21[["TOTAL MX"]]
Var_Dep= gastos21["TIPO GASTO"]

In [30]:
#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

In [31]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

In [32]:
#Se escalan todos los datos
escalar = StandardScaler()

In [33]:
#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [34]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [35]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [36]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 
y_pred

array(['COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'OTROS GASTOS', 'COMPRAS', 'COMPRAS', 'OTROS GASTOS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'COMPRAS', 'OTROS GASTOS', 'OTROS GASTOS',
       'COMPRAS', 'OTROS GASTOS', 'COMPRAS', 'OTROS GASTOS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'OTROS GASTOS',
       'COMPRAS', 'OTROS GASTOS', 'OTROS GASTOS', 'OTROS GASTOS',
       'COMPRAS', 'COMPRAS', 'OTROS GASTOS', 'COMPRAS', 'OTROS GASTOS',
       'COMPRAS', 'OTROS GASTOS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'OTROS GASTOS', 'OTROS GASTOS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'COMPRAS', 'OTROS GASTOS', 'COMPRAS',
       'OTROS GASTOS', 'OTROS GASTOS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'COMPRAS', 'OTROS GASTOS', 'COMPRAS',
       'OTROS GASTOS', 'OTROS GASTOS', 'COMPRAS', 'COMPRAS',
       'OTROS GASTOS', 'COM

In [37]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[309 183]
 [297 122]]


In [38]:
#Calculo la precisión del modelo - OTROS GASTOS
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="OTROS GASTOS")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.4


In [39]:
#Calculo la precisión del modelo - COMPRAS
from sklearn.metrics import precision_score

precision2 = precision_score(y_test, y_pred, average="binary", pos_label="COMPRAS")
print('Precisión del modelo:')
print(precision2)

Precisión del modelo:
0.5099009900990099


In [40]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.47310647639956094


In [41]:
#Calculo la sensibilidad del modelo - OTROS GASTOS
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="OTROS GASTOS")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.2911694510739857


In [42]:
#Calculo la sensibilidad del modelo - COMPRAS
from sklearn.metrics import recall_score

sensibilidad2 = recall_score(y_test, y_pred, average="binary", pos_label="COMPRAS")
print('Sensibilidad del modelo:')
print(sensibilidad2)

Sensibilidad del modelo:
0.6280487804878049
